In [2]:
from torch import optim
from torch import nn
import torch
import torch.nn.functional as F
import torchvision

import matplotlib.pyplot as plt
import numpy as np

import torchvision.transforms as transforms


In [3]:
# https://pytorch.org/vision/stable/generated/torchvision.datasets.CIFAR10.html
# https://www.cs.toronto.edu/~kriz/cifar.html
train_dataset = torchvision.datasets.CIFAR10(root='cifar10/',
                                             train=True,  
                                             transform=transforms.ToTensor(), 
                                             download=True)
#train_dataset=train_dataset[0:1000,...]

Files already downloaded and verified


In [26]:
cnt=0
train_size=3000
short_train_set=[]
for (image,target) in train_dataset:
    #print ("t=",target)
    image = image.permute(1, 2, 0)
    short_train_set.append((image,target))
    #plt.imshow(image)
    #plt.show()
    cnt+=1
    if cnt>train_size:
        break

In [27]:
# https://pytorch.org/docs/stable/data.html#map-style-datasets
train_loader = torch.utils.data.DataLoader(dataset=short_train_set,
                                           batch_size=100, 
                                           shuffle=True)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [28]:
class Perceptron(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, hidden_layer_quantity):
        super().__init__()
        self.layers = []
        self.layers.append(nn.Linear(input_size,hidden_size))
        for cnt in range(0,hidden_layer_quantity):
            self.layers.append(nn.Linear(hidden_size,hidden_size))
        self.layers.append(nn.Linear(hidden_size,output_size))
        for cnt,layer in enumerate(self.layers):
            key = "layer{}".format(cnt)
            self.add_module(key,layer)

    def flat_x(self,x):
        # Changing the input matrix to the vector view
        new_x = x.view(x.shape[0], -1)
        return new_x

    def forward(self, x):
        xtr = self.flat_x(x)
        for cnt,layer in enumerate(self.layers):
            if cnt<len(self.layers)-1:
                xtr = F.leaky_relu(layer(xtr))
            else :
                xtr = F.sigmoid(layer(xtr))
        return xtr
    
    def predict(self, x):
        xtr = self.forward(x)
        xtr = F.softmax(xtr)
        return xtr

perceptron = Perceptron(3072,128,10,1)
#perceptron
for idx, m in enumerate(perceptron.modules()):
     print(idx, '->', m)

0 -> Perceptron(
  (layer0): Linear(in_features=3072, out_features=128, bias=True)
  (layer1): Linear(in_features=128, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=10, bias=True)
)
1 -> Linear(in_features=3072, out_features=128, bias=True)
2 -> Linear(in_features=128, out_features=128, bias=True)
3 -> Linear(in_features=128, out_features=10, bias=True)


In [29]:
criteria = nn.MSELoss()
optimizer = optim.SGD(perceptron.parameters(), lr=0.05, momentum=0.01)

In [31]:
num_epochs = 1000
from tensorflow.keras.utils import to_categorical

for epoch in range(num_epochs):
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        x_train, y_train = data[0], data[1]
        y_train = F.one_hot(num_classes=10,input=y_train).float()

        #print (y_train)
        #print (x_train.shape)
        optimizer.zero_grad()
        y_pred = perceptron(x_train)
        loss = criteria(y_pred, y_train)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print("Epoch [{}/{}] loss: {}".format(epoch,num_epochs,running_loss))            

print('Training is finished!')

Epoch [0/1000] loss: 2.0119195580482483
Epoch [1/1000] loss: 1.9408010393381119
Epoch [2/1000] loss: 1.9345795661211014
Epoch [3/1000] loss: 1.9924582354724407
Epoch [4/1000] loss: 2.067286267876625
Epoch [5/1000] loss: 1.9748672023415565
Epoch [6/1000] loss: 1.9079247657209635
Epoch [7/1000] loss: 1.8877461506053805
Epoch [8/1000] loss: 1.9559864029288292
Epoch [9/1000] loss: 1.8904834892600775
Epoch [10/1000] loss: 1.9310527704656124
Epoch [11/1000] loss: 1.956943567842245
Epoch [12/1000] loss: 1.8762936287093908
Epoch [13/1000] loss: 1.9116580002009869
Epoch [14/1000] loss: 1.9388471599668264
Epoch [15/1000] loss: 1.9631859101355076
Epoch [16/1000] loss: 1.9983665011823177
Epoch [17/1000] loss: 1.9767513498663902
Epoch [18/1000] loss: 1.9450512006878853
Epoch [19/1000] loss: 1.9425898306071758
Epoch [20/1000] loss: 1.910779234021902
Epoch [21/1000] loss: 1.9549425318837166
Epoch [22/1000] loss: 1.8886921033263206
Epoch [23/1000] loss: 1.8604663128498942
Epoch [24/1000] loss: 1.87278